In [ ]:
import requests
import json
import pandas as pd
import hashlib # hash 함수용 sha256 사용할 라이브러리|
import random

DApp 제작 -1 계산기

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {
                           "contract_code" : "calculate_result = {}{}{}"}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
# 예시: 120 + 360
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] ==contract_address:
            exec( _tx['smart_contract']['contract_code'].format(120,"+",360))
            break
print(calculate_result) 

In [ ]:
# 3 X 5
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] ==contract_address:
            exec( _tx['smart_contract']['contract_code'].format(3,"*",5))
            break
print(calculate_result)        

DApp 제작 -2 복권

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "smart_contract",
        "amount": 0,
        "smart_contract": {
                           "contract_code" : """
def Lottery():
    lottery_number = random.sample(range(1,46),6)
    lottery_number = sorted(lottery_number, key=lambda x: x)
    lottery_number
    print(lottery_number) 
    return lottery_number
                           """}
}
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']

headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
# 실제로 6개 번호 선정하는 command
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] ==contract_address:
            exec( _tx['smart_contract']['contract_code'])
            break   
print(Lottery())   

DApp 제작 -3 Defi

In [ ]:
token_name = 'pyTOKEN' 
token_total_volume = 100000
token_owner = {'token_maker' : 10000}
staking_status = {}

def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]

def send_token(sender,recipent,amount):
    if sender in token_owner.keys():
        if get_balance(sender) > amount:
            token_owner[sender]  = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent]  = token_owner[recipent] + amount
            else :
                token_owner[recipent]  =  amount
            print("Transaction Completed")
            get_balance(sender) 
            get_balance(recipent) 

        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Sender id"
    
def token_staking(staker,amount):
    if staker in token_owner.keys():
        if get_balance(staker) > amount:
            token_owner[staker]  = token_owner[staker] - amount
            staking_status [len(staking_status)] =  {'staker':staker,'amount':amount}
            print("Staing Completed")
            get_balance(staker) 
            
        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Staker id"
    
def staking_yield(staking_status):
    for t in staking_status:
        print(staking_status[t])
        staking_status[t]['amount'] = staking_status[t]['amount'] * 1.1
    return staking_status

In [ ]:
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
        "sender": "test_from",
        "recipient": "test_to",
        "amount": 3,
        "smart_contract": {
                           "contract_code" :"token_name = 'pySTAKINGTOKEN' \ntoken_total_volume = 100000\ntoken_owner = {'token_maker' : 10000}\nstaking_status = {}",
                           "contract_function_getBalance" :"""
def get_balance(user_id):
    print('{} Balance is : '.format(user_id), token_owner[user_id])
    return token_owner[user_id]
""",
                           "contract_function_sendToken" :"""
def send_token(sender,recipent,amount):
    if sender in token_owner.keys():  
        if get_balance(sender) > amount:
            token_owner[sender]  = token_owner[sender] - amount
            if recipent in token_owner.keys():
                token_owner[recipent]  = token_owner[recipent] + amount
            else :
                token_owner[recipent]  =  amount
            print("Transaction Completed")
            get_balance(sender) 
            get_balance(recipent) 

        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Sender id"
""",
                           "contract_function_token_staking" :"""
def token_staking(staker,amount):
    if staker in token_owner.keys():                       ## 예치자(staker)가 실제 존재하는 사용자인지 확인
        if get_balance(staker) > amount:                   ## 예치자(staker)의 잔고가 예치 금액보다 많은지 확인
            token_owner[staker]  = token_owner[staker] - amount   ## 예치자(staker)의 잔고에서 예치 금액 제외
            staking_status [len(staking_status)] =  {'staker':staker,'amount':amount}  
            ## 예치 정보(staking_status)에 예치자(staker)의 예치내역 저장
            print("Staing Completed")
            get_balance(staker) 
            
        else:
            return "Insufficient Balance"
    else:
        return "Unavailable Staker id"
""",
                           "contract_function_staking_yield" :"""
def staking_yield(staking_status):                                 ## 예치 이자 지급함수
    for t in staking_status:
        print(staking_status[t])
            staking_status[t]['amount'] = staking_status[t]['amount'] * (1+0.1)    ## 예치 이자가 10% 지급된 금액으로 예치금 변경
    return staking_status
"""                       

                                            }
            }
result = requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content
contract_address = json.loads(result)['contract_address']


headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

In [ ]:
# pySTAKINGTOKEN 생성여부 확인
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
res_json = json.loads(res.content)

for _block in res_json['chain']:
    for _tx in _block['transactions']:
        if _tx['smart_contract']['contract_address'] == contract_address:
            exec( _tx['smart_contract']['contract_code'])
            break

In [ ]:
# pySTAKINGTOKEN 예치 진행
exec(_tx['smart_contract']['contract_function_token_staking'])
token_staking('token_maker',100)

In [ ]:
# pySTAKINGTOKEN 예치자에게 이자 지급
exec(_tx['smart_contract']['contract_function_staking_yield'])
staking_yield(staking_status)